In [1]:
import numpy as np
np.set_printoptions(threshold=np.inf)
import nibabel as nib
import matplotlib.pyplot as plt
import os
import math
import time
from nilearn import image
from nilearn.plotting import show
from nilearn.plotting import plot_roi
from nilearn.masking import compute_background_mask
from nilearn import plotting
from nilearn.masking import intersect_masks
from nilearn.masking import apply_mask
from nilearn.masking import unmask

C:\Users\16609\AppData\Roaming\Python\Python36\site-packages\nilearn\__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


In [2]:
def get_mask(img):
    img_smthed = image.smooth_img(img, fwhm='fast')
    img_mask = compute_background_mask(img_smthed, border_size=0, connected=True, opening=True)
    return img_mask

#read batch nii files and get every mask of them
def mask_nii(filepath, masks_path):
    #the original .nii is stored in /filepath
    filenames = os.listdir(filepath)
    #filenames = os.listdir(filepath).remove('.DS_Store')
    i = 0
    for f in filenames:
        #load the original .nii file
        img_path = os.path.join(filepath, f)
        print(i)
        if f == ".DS_Store":
            continue
        print(f)
        i += 1
        img = nib.load(img_path)
        img = img.slicer[1:-1,:,:-1]
        img = img.slicer[1::2,1::2,1::2]
        # get the mask of the .nii file
        img_mskd = get_mask(img)
        # save every mask to 'masks_path'
        img_mskd.to_filename(os.path.join(masks_path, f))

# get the intersection of the masks
def mask_merge(masks_path):
    # all the masks are stored in 'masks_path'
    filenames = os.listdir(masks_path)
    newlist = []
    for f in filenames:
        # load the mask
        img_path = os.path.join(masks_path, f)
        # get a list of the locations of masks
        newlist += [img_path]
    # get the intersect of these masks
    intersect = intersect_masks(mask_imgs=newlist, threshold=0.5, connected=True)
    intersect.to_filename(os.path.join(masks_path, 'intersect.nii.gz'))
    return intersect

#apply the merged mask to the original .nii files
def apply_mask_func(filepath, masked_path, intersect):
    filenames = os.listdir(filepath)
    # with open('../MRI dataset/masked_data.txt','ab') as g:
    for f in filenames:# load the original .nii image
        img_path = os.path.join(filepath, f)
        img = nib.load(img_path)
        # apply the mask on these images
        msked_img = apply_mask(img, intersect)
        # save the masked data as .npy file (location:masked_path)
        save_f_name = f.replace('.nii.gz', '.nii')
        # save_f_path = os.path.join(masked_path, save_f_name+'.npy')
        # np.save(file=save_f_path, arr=msked_img)  ##save individual img
        msked_img.tofile(os.path.join(masked_path, save_f_name))
        ####if we want to save the data in .txt
            #np.savetxt(g, msked_img, delimiter=" ")

In [3]:
filepath = "./triple-crossfitting/waiting_mask/"
masks_path = "./triple-crossfitting/cc_masks0/"
masked_path = './triple-crossfitting/cc_masked/'

In [15]:
mask_nii(filepath, masks_path)  ##get the smoothed masks of every original image
intersect = mask_merge(masks_path)   # get the intersection of all the masks
# apply_mask_func(filepath, masked_path, intersect)  ## apply the intersection to every original mask
# plot_roi(intersect, bg_img=None)
# intersect = nib.load(masks_path + "intersect.nii.gz")

0
ADNIADsim+002_S_0619+AD-m00-1.5T.nii.gz
1
ADNIADsim+002_S_0816+AD-m00-1.5T.nii.gz
2
ADNIADsim+002_S_0938+AD-m00-1.5T.nii.gz
3
ADNIADsim+002_S_0955+AD-m00-1.5T.nii.gz
4
ADNIADsim+002_S_1018+AD-m00-1.5T.nii.gz
5
ADNIADsim+003_S_1059+AD-m00-1.5T.nii.gz
6
ADNIADsim+003_S_1257+AD-m00-1.5T.nii.gz
7
ADNIADsim+005_S_0221+AD-m00-1.5T.nii.gz
8
ADNIADsim+005_S_0814+AD-m00-1.5T.nii.gz
9
ADNIADsim+005_S_0929+AD-m00-1.5T.nii.gz
10
ADNIADsim+005_S_1341+AD-m00-1.5T.nii.gz
11
ADNIADsim+006_S_0547+AD-m00-1.5T.nii.gz
12
ADNIADsim+007_S_0316+AD-m00-1.5T.nii.gz
13
ADNIADsim+007_S_1304+AD-m00-1.5T.nii.gz
14
ADNIADsim+007_S_1339+AD-m00-1.5T.nii.gz
15
ADNIADsim+009_S_1334+AD-m00-1.5T.nii.gz
16
ADNIADsim+009_S_1354+AD-m00-1.5T.nii.gz
17
ADNIADsim+010_S_0786+AD-m00-1.5T.nii.gz
18
ADNIADsim+010_S_0829+AD-m00-1.5T.nii.gz
19
ADNIADsim+011_S_0003+AD-m00-1.5T.nii.gz
20
ADNIADsim+011_S_0010+AD-m00-1.5T.nii.gz
21
ADNIADsim+011_S_0053+AD-m00-1.5T.nii.gz
22
ADNIADsim+011_S_0183+AD-m00-1.5T.nii.gz
23
ADNIADsim+012_S_06

189
ADNIMCIsim+007_S_0041+MCI-m00-1.5T.nii.gz
190
ADNIMCIsim+007_S_0101+MCI-m00-1.5T.nii.gz
191
ADNIMCIsim+007_S_0128+MCI-m00-1.5T.nii.gz
192
ADNIMCIsim+007_S_0249+MCI-m00-1.5T.nii.gz
193
ADNIMCIsim+007_S_0293+MCI-m00-1.5T.nii.gz
194
ADNIMCIsim+007_S_0344+MCI-m00-1.5T.nii.gz
195
ADNIMCIsim+007_S_0414+MCI-m00-1.5T.nii.gz
196
ADNIMCIsim+007_S_0698+MCI-m00-1.5T.nii.gz
197
ADNIMCIsim+009_S_1030+MCI-m00-1.5T.nii.gz
198
ADNIMCIsim+010_S_0161+MCI-m00-1.5T.nii.gz
199
ADNIMCIsim+010_S_0422+MCI-m00-1.5T.nii.gz
200
ADNIMCIsim+010_S_0904+MCI-m00-1.5T.nii.gz
201
ADNIMCIsim+011_S_0168+MCI-m00-1.5T.nii.gz
202
ADNIMCIsim+011_S_0241+MCI-m00-1.5T.nii.gz
203
ADNIMCIsim+011_S_0326+MCI-m00-1.5T.nii.gz
204
ADNIMCIsim+011_S_0362+MCI-m00-1.5T.nii.gz
205
ADNIMCIsim+011_S_0856+MCI-m00-1.5T.nii.gz
206
ADNIMCIsim+011_S_0861+MCI-m00-1.5T.nii.gz
207
ADNIMCIsim+011_S_1080+MCI-m00-1.5T.nii.gz
208
ADNIMCIsim+011_S_1282+MCI-m00-1.5T.nii.gz
209
ADNIMCIsim+012_S_0634+MCI-m00-1.5T.nii.gz
210
ADNIMCIsim+012_S_0917+MCI-m00-

370
ADNIMCIsim+052_S_1352+MCI-m00-1.5T.nii.gz
371
ADNIMCIsim+053_S_0389+MCI-m00-1.5T.nii.gz
372
ADNIMCIsim+053_S_0507+MCI-m00-1.5T.nii.gz
373
ADNIMCIsim+053_S_0621+MCI-m00-1.5T.nii.gz
374
ADNIMCIsim+053_S_0919+MCI-m00-1.5T.nii.gz
375
ADNIMCIsim+057_S_0464+MCI-m00-1.5T.nii.gz
376
ADNIMCIsim+057_S_0839+MCI-m00-1.5T.nii.gz
377
ADNIMCIsim+057_S_0941+MCI-m00-1.5T.nii.gz
378
ADNIMCIsim+057_S_0957+MCI-m00-1.5T.nii.gz
379
ADNIMCIsim+057_S_1007+MCI-m00-1.5T.nii.gz
380
ADNIMCIsim+057_S_1217+MCI-m00-1.5T.nii.gz
381
ADNIMCIsim+057_S_1265+MCI-m00-1.5T.nii.gz
382
ADNIMCIsim+057_S_1269+MCI-m00-1.5T.nii.gz
383
ADNIMCIsim+062_S_1182+MCI-m00-1.5T.nii.gz
384
ADNIMCIsim+062_S_1294+MCI-m00-1.5T.nii.gz
385
ADNIMCIsim+062_S_1299+MCI-m00-1.5T.nii.gz
386
ADNIMCIsim+067_S_0038+MCI-m00-1.5T.nii.gz
387
ADNIMCIsim+067_S_0045+MCI-m00-1.5T.nii.gz
388
ADNIMCIsim+067_S_0077+MCI-m00-1.5T.nii.gz
389
ADNIMCIsim+067_S_0098+MCI-m00-1.5T.nii.gz
390
ADNIMCIsim+067_S_0176+MCI-m00-1.5T.nii.gz
391
ADNIMCIsim+067_S_0243+MCI-m00-

549
ADNIMCIsim+941_S_1311+MCI-m00-1.5T.nii.gz
550
ADNINORMALsim+002_S_0295+NORMAL-m00-1.5T.nii.gz
551
ADNINORMALsim+002_S_0413+NORMAL-m00-1.5T.nii.gz
552
ADNINORMALsim+002_S_0559+NORMAL-m00-1.5T.nii.gz
553
ADNINORMALsim+002_S_0685+NORMAL-m00-1.5T.nii.gz
554
ADNINORMALsim+002_S_1261+NORMAL-m00-1.5T.nii.gz
555
ADNINORMALsim+002_S_1280+NORMAL-m00-1.5T.nii.gz
556
ADNINORMALsim+003_S_0907+NORMAL-m00-1.5T.nii.gz
557
ADNINORMALsim+003_S_0931+NORMAL-m00-1.5T.nii.gz
558
ADNINORMALsim+003_S_0981+NORMAL-m00-1.5T.nii.gz
559
ADNINORMALsim+003_S_1021+NORMAL-m00-1.5T.nii.gz
560
ADNINORMALsim+005_S_0553+NORMAL-m00-1.5T.nii.gz
561
ADNINORMALsim+005_S_0602+NORMAL-m00-1.5T.nii.gz
562
ADNINORMALsim+005_S_0610+NORMAL-m00-1.5T.nii.gz
563
ADNINORMALsim+006_S_0484+NORMAL-m00-1.5T.nii.gz
564
ADNINORMALsim+006_S_0498+NORMAL-m00-1.5T.nii.gz
565
ADNINORMALsim+006_S_0681+NORMAL-m00-1.5T.nii.gz
566
ADNINORMALsim+006_S_0731+NORMAL-m00-1.5T.nii.gz
567
ADNINORMALsim+007_S_0068+NORMAL-m00-1.5T.nii.gz
568
ADNINORMALsim+

708
ADNINORMALsim+114_S_0173+NORMAL-m00-1.5T.nii.gz
709
ADNINORMALsim+114_S_0416+NORMAL-m00-1.5T.nii.gz
710
ADNINORMALsim+114_S_0601+NORMAL-m00-1.5T.nii.gz
711
ADNINORMALsim+116_S_0382+NORMAL-m00-1.5T.nii.gz
712
ADNINORMALsim+116_S_0648+NORMAL-m00-1.5T.nii.gz
713
ADNINORMALsim+116_S_0657+NORMAL-m00-1.5T.nii.gz
714
ADNINORMALsim+116_S_1232+NORMAL-m00-1.5T.nii.gz
715
ADNINORMALsim+116_S_1249+NORMAL-m00-1.5T.nii.gz
716
ADNINORMALsim+123_S_0072+NORMAL-m00-1.5T.nii.gz
717
ADNINORMALsim+123_S_0106+NORMAL-m00-1.5T.nii.gz
718
ADNINORMALsim+123_S_0113+NORMAL-m00-1.5T.nii.gz
719
ADNINORMALsim+123_S_0298+NORMAL-m00-1.5T.nii.gz
720
ADNINORMALsim+126_S_0405+NORMAL-m00-1.5T.nii.gz
721
ADNINORMALsim+126_S_0605+NORMAL-m00-1.5T.nii.gz
722
ADNINORMALsim+126_S_0680+NORMAL-m00-1.5T.nii.gz
723
ADNINORMALsim+127_S_0259+NORMAL-m00-1.5T.nii.gz
724
ADNINORMALsim+127_S_0260+NORMAL-m00-1.5T.nii.gz
725
ADNINORMALsim+127_S_0622+NORMAL-m00-1.5T.nii.gz
726
ADNINORMALsim+127_S_0684+NORMAL-m00-1.5T.nii.gz
727
ADNINORM

In [9]:
t_grid = 180010
L = 300
grid = math.ceil(t_grid / 300)
last_grid = t_grid - grid * (L-1)
img_array = np.zeros(shape=[L, grid])
w_masked_path = './triple-crossfitting/cc_masks0_b/'
filename_lst = [w_masked_path+i for i in os.listdir(w_masked_path)]
for img_n in filename_lst:
    if img_n == 'intersect.nii.gz':
        continue
    img_id = img_n.split('/')[-1].split('__')[0]
    img = nib.load(img_n)
    a = apply_mask(img, intersect)
    for l in range(L):
        if l != L-1:
            img_array[l] = a[l*grid:(l+1)*grid]
        else:
            img_array[l][:last_grid] = a[-last_grid:]
    np.save('./triple-crossfitting/total_image_array/'+img_id+'-img_part.npy', img_array)